In [1]:
CLUSTER_TYPE ='docker_cluster'

In [13]:
import os

KAFKA_BOOTSTRAP_SERVERS = ''

if CLUSTER_TYPE == 'local':

    KAFKA_HOME = '~/Downloads/kafka_2.13-2.8.0'
    KAFKA_BOOTSTRAP_SERVERS = ['localhost:9092']
    
    # Start Zookeeper    
    os.system('{0}/bin/zookeeper-server-start.sh {0}/config/zookeeper.properties'.format(KAFKA_HOME)) 
    
    # Start one Kafka Broker
    os.system('{0}/bin/kafka-server-start.sh {0}/config/server.properties'.format(KAFKA_HOME)) 
    
elif CLUSTER_TYPE == 'docker_cluster':

    KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'

In [14]:
import csv
import dask.array as da
import numpy as np
from time import sleep
import threading
rexp = np.random.exponential

def data_post(file,rate,log):
    with open(file,newline = '') as file:
        #times = da.random.exponential(1/rate,12000).compute()
        data = csv.DictReader(file)
        for i, entry in enumerate(data):
            #sleep(times[i])
            sleep(max(rexp(1/rate)-0.00050,0))# a lot of time is wasted elsewhere, "one by one" rates are hard to achieve
            #x = threading.Thread(target = log, args = (entry,))
            #x.start()
            log(entry)

In [15]:
%%time
sample_file = 'sample_data.csv'
entries_per_sec = 300

data_post(sample_file, entries_per_sec, lambda x: None)

CPU times: user 1.5 s, sys: 226 ms, total: 1.73 s
Wall time: 37.7 s


In [16]:
from kafka import KafkaProducer
import json

In [17]:
producer = KafkaProducer(
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
                         value_serializer=lambda x: json.dumps(x).encode('utf-8')
                        )

In [18]:
topic = 'raw-data'
def logger(x):
    y = {
        "HEAD": int(x['HEAD']),
        "FPGA": int(x['FPGA']),
        "TDC_CHANNEL": int(x['TDC_CHANNEL']),
        "ORBIT_CNT": int(x['ORBIT_CNT']),
        "BX_COUNTER": int(x['BX_COUNTER']),
        "TDC_MEAS": float(x['TDC_MEAS'])
    }
    producer.send(topic, value = y)
    producer.flush()


In [19]:
%%time
data_post(sample_file, entries_per_sec, logger)

CPU times: user 35.2 s, sys: 3.69 s, total: 38.9 s
Wall time: 1min 19s
